In [ ]:
# standard imports
import numpy as np
import math
import scipy
import os
import csv
from tqdm import tqdm
import meshio
import matplotlib.pyplot as plt

In [ ]:
# custom class imports
from node import node
from cell import cell
from mesh import exp_mesh

# derivative methods
from derivatives import *

# utility functions
from utilities import *

# Setting Problem

In [ ]:
# read su2 mesh file
meshio_mesh = meshio.read("./lam_cyl_comp_air/mesh_cylinder_quad.su2",file_format="su2")
# define mid point for rotational symmetric mesh
mid_point = np.array([0.5,0]) 
# convert mesh into custom class exp_mesh
c_mesh = exp_mesh(meshio_mesh,mid_point)

# Loading Filedata

In [ ]:
# reading file data
data_dir="./lam_cyl_comp_air/restart/"
data_dict = get_flow_data(data_dir,1000,1,180)

# Data Preparation

In [ ]:
# configure state data 
q = np.vstack([data_dict["u"],data_dict["v"],data_dict["a"]])
q_avg = np.mean(q,1)
q_fluc = q - np.repeat(np.expand_dims(q_avg,1),q.shape[1],axis=1)

# Operator Configuration

In [ ]:
n = c_mesh.n
N = c_mesh.N

In [ ]:
def Q(mesh,q1,q2,method="fd"):
    gamma = 1.4
    ## methods for node based calculation
    # fd -> finite differences
    # pd -> polynomial approximation
    ## methods for cell based calculation
    # bm -> bilinear face interpolation
    
    d = int(len(q1)/3)
    
    # decomposition into variable vectors
    u1 = q1[:d]
    v1 = q1[d:2*d]
    a1 = q1[2*d:3*d]
    u2 = q2[:d]
    v2 = q2[d:2*d]
    a2 = q2[2*d:3*d]

    # initializing spatial derivatives
    u2x = np.empty(d)
    v2x = np.empty(d)
    a2x = np.empty(d)
    u2y = np.empty(d)
    v2y = np.empty(d)
    a2y = np.empty(d)

    # computation of spatial derivatives
    if len(q1==mesh.n):
        if method=="fd":
            finite_differences(mesh,u2)
        elif method=="pd":
            polynomial_derivatives(mesh,u2)
        else:
            raise ValueError("Invalid for node based derivatives!")
        for nod in mesh.nodes:
            u2x[nod.index]=nod.dx
            u2y[nod.index]=nod.dy
        
        if method=="fd":
            finite_differences(mesh,v2)
        elif method=="pd":
            polynomial_derivatives(mesh,v2)
        for nod in mesh.nodes:
            v2x[nod.index]=nod.dx
            v2y[nod.index]=nod.dy

        if method=="fd":
            finite_differences(mesh,a2)
        elif method=="pd":
            polynomial_derivatives(mesh,a2)    
        for nod in mesh.nodes:
            a2x[nod.index]=nod.dx
            a2y[nod.index]=nod.dy
    elif len(q1==mesh.N):
        if method!="bm":
            raise ValueError("Selected method not applicaple to cell based data! Please select bilinear mapping!")
        bilinear_derivatives(mesh,u2)
        for nod in mesh.nodes:
            u2x[nod.index]=nod.dx
            u2y[nod.index]=nod.dy
        bilinear_derivatives(mesh,v2)
        for nod in mesh.nodes:
            v2x[nod.index]=nod.dx
            v2y[nod.index]=nod.dy
        bilinear_derivatives(mesh,a2)
        for nod in mesh.nodes:
            a2x[nod.index]=nod.dx
            a2y[nod.index]=nod.dy
    else:
        raise ValueError("Invalid dimension for input data!")
    
    # computation of index based operator
    u_tmp = np.multiply(u1,u2x) + np.multiply(v1,u2y) + 2/(gamma - 1)*np.multiply(a1,a2x)
    v_tmp = np.multiply(u1,v2x) + np.multiply(v1,v2y) + 2/(gamma - 1)*np.multiply(a1,a2y)
    a_tmp = np.multiply(u1,a2x) + np.multiply(v1,a2y) + (gamma - 1)/2*np.multiply(a1,np.add(u2x,u2y))
    return np.concatenate((u_tmp,v_tmp,a_tmp))

In [ ]:
def L(mesh,q,method="fd"):
    # fd -> finite differences
    # pd -> polynomial approximation   
    d = int(len(q)/3)
    
    # decomposition into variable vectors
    u = q[:d]
    v = q[d:2*d]
    a = q[2*d:3*d]    

    if len(q)!=mesh.n:
        ValueError("Invalid data dimension!")
    
    # initializing spatial derivatives
    uxx = np.empty(d)
    uyy = np.empty(d)
    vxx = np.empty(d)
    vyy = np.empty(d)
    
    # computing second derivatives
    if method=="fd":
        finite_differences(mesh,u,second=True)
    elif method=="pd":
        polynomial_derivatives(mesh,u,second=True)
    else:
        raise ValueError("Invalid for node based derivatives!")
    for nod in mesh.nodes:
        uxx[nod.index]=nod.ddx
        uyy[nod.index]=nod.ddy
    
    if method=="fd":
        finite_differences(mesh,v,second=True)
    elif method=="pd":
        polynomial_derivatives(mesh,v,second=True)
    for nod in mesh.nodes:
        vxx[nod.index]=nod.ddx
        vyy[nod.index]=nod.ddy

    # computation of index based operator
    u_tmp = np.add(uxx,uyy)
    v_tmp = np.add(vxx,vyy)
    a_tmp = np.zeros(d)
    return np.concatenate((u_tmp,v_tmp,a_tmp))

In [ ]:
def L2(mesh,q,method="fd",output="node"):
    # fd -> finite differences
    # pd -> polynomial approximation   
    d = int(len(q)/3)
    
    # decomposition into variable vectors
    u = q[:d]
    v = q[d:2*d]

    if len(q)!=mesh.n:
        ValueError("Invalid data dimension!")
    
    # initializing spatial derivatives
    ulap = np.empty(d)
    vlap = np.empty(d)

    # computing second derivatives
    if method=="fd":
        finite_differences(mesh,u,second=True)
    elif method=="pd":
        polynomial_derivatives(mesh,u,second=True)
    else:
        raise ValueError("Invalid for node based derivatives!")
    for nod in mesh.nodes:
        ulap[nod.index]=nod.laplacian
    
    if method=="fd":
        finite_differences(mesh,v,second=True)
    elif method=="pd":
        polynomial_derivatives(mesh,v,second=True)
    for nod in mesh.nodes:
        vlap[nod.index]=nod.laplacian

    if output=="cell":
        ulap = mesh.compute_cell_values_from_node_data(ulap)
        vlap = mesh.compute_cell_values_from_node_data(vlap)
        d = int(mesh.N/3)

    # computation of index based operator
    u_tmp = ulap # np.add(uxx,uyy)
    v_tmp = vlap
    a_tmp = np.zeros(d)
    return np.concatenate((u_tmp,v_tmp,a_tmp))

In [ ]:
def inner_product(mesh,q1,q2):
    ## energy based inner product
    integral = 0    
    alpha = 1
    gamma = 1.4
    mach_weight = 2 * alpha / (gamma - 1)
    
    d = int(len(q1)/3)
    f = int(len(q2)/3)

    # case node data
    if d==mesh.n and f==mesh.n:
        for nod in mesh.nodes:
            i = nod.index
            # summation over nodes with corresponding node volume 
            integral += (q1[i]*q2[i]+q1[2*i]*q2[2*i]+mach_weight*q1[3*i]*q2[3*i]) * nod.dv
        return integral
    # case cell data
    elif d==mesh.N and f==mesh.N:
        for cel in mesh.cells:
            i = cel.index
            # summation over ncells with corresponding cell volumes
            integral += (q1[i]*q2[i]+q1[2*i]*q2[2*i]+mach_weight*q1[3*i]*q2[3*i]) * cel.volume
        return integral
    # case invalid dimensions
    else:
        raise ValueError("Invalid Dimension in data vectors!")
    return 0

In [ ]:
test = L2(c_mesh,q_fluc[:,0],method="fd")[:400]

In [ ]:
np.isnan(test).any()